# We start off by importing the data

In [ ]:
import pandas as pd
import plotly.graph_objects as go
df = pd.read_pickle('data/vac-prep.pkl')
pd.options.plotting.backend = "plotly"

## Let's take a look

In [ ]:
df.tail(10)

In [ ]:
current_official_doses = df.tail(1)['dosen_kumulativ']
#current_official_doses.values[0]
current_official_doses





In [ ]:
df.set_index('date')['dosen_kumulativ'].plot.line(width=1000, height=500, title='Impfungen Bayern').show()

### Average doses of the last days

In [ ]:
a = pd.DataFrame(columns=["Last x days", "Average vacs"], data=[
    [3, df.tail(3)['vortag_dosen_dif'].mean()],
    [5, df.tail(5)['vortag_dosen_dif'].mean()],
    [7, df.tail(7)['vortag_dosen_dif'].mean()],
    [10, df.tail(10)['vortag_dosen_dif'].mean()],
    [14, df.tail(14)['vortag_dosen_dif'].mean()],
])

fig = go.Figure(data=[
    go.Bar(name='Three',x=a['Last x days'], y=a['Average vacs'])

])
fig.update_layout(
    width=500,
    height=400,
    title='Average doses of the last x days'
)
fig.show()

In [ ]:
a

In [ ]:
df['rolling_vortag_dosen_dif'] = df.vortag_dosen_dif.rolling(7).mean()

to_plot = df.set_index('date')

fig = go.Figure()
fig.add_trace(go.Scatter(x = to_plot.index,
                         y=to_plot.vortag_dosen_dif,
                         mode='markers',
                         name='Real',
                         line=dict(color='grey', width=1)))
fig.add_trace(go.Scatter(x = to_plot.index,
                         y=to_plot.rolling_vortag_dosen_dif,
                         mode='lines',
                         name='Smoothed',
                         line=dict(color='cornflowerblue', width=4)))

fig.update_layout(
    width=1000,
    height=500,
    title='Daily Vaccination speed change'
)
fig.show()

## Vaccinations per weekday

In [ ]:

to_plot_all = df.groupby(["weekday_name"])['vortag_dosen_dif'].mean().sort_values()
to_plot_10 = df.tail(7*10).groupby(["weekday_name"])['vortag_dosen_dif'].mean().sort_values()
to_plot_5 = df.tail(7*5).groupby(["weekday_name"])['vortag_dosen_dif'].mean().sort_values()
to_plot_2 = df.tail(7*2).groupby(["weekday_name"])['vortag_dosen_dif'].mean().sort_values()


fig = go.Figure(data=[
    go.Bar(name='complete timeline', y=to_plot_all, x=to_plot_all.index),
    go.Bar(name='10 weeks', y=to_plot_10, x=to_plot_all.index),
    go.Bar(name='5 weeks', y=to_plot_5, x=to_plot_all.index),
    go.Bar(name='2 weeks', y=to_plot_2, x=to_plot_all.index),
])
fig.update_layout(
    width=1000,
    height=500,
    title='Average vaccinations per weekday'
)
fig.show()

## Vaccinations per calendar week

In [ ]:
# tail(length - 5) removes 53rd calendar week
to_plot_sum = df.tail(len(df.index) - 8).groupby(["calendar_week"])['vortag_dosen_dif'].sum()
to_plot_mean = df.tail(len(df.index) - 8).groupby(["calendar_week"])['vortag_dosen_dif'].mean()

fig = go.Figure(data=[
    go.Bar(name='Sum', y=to_plot_sum),
    go.Bar(name='Mean', y=to_plot_mean),
])
fig.update_layout(
    width=1000,
    height=500,
    title='Vaccinations per calendar week'
)
fig.show()


